# Данные

In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

Читаем мета-файл (дважды, потому что один из них будем менять, а второй референсить). Можно было бы сделать более логично, но я почему-то решила так.

In [2]:
with open('../preprocess/meta.json', 'r') as f:
    meta = json.load(f)

with open('../preprocess/meta.json', 'r') as f:
    original_meta = json.load(f)

Загружаем файл с обработанными данными, заменяем все пустые ячейки на `none`. Откуда пустые ячейки? Они возникают в вариантах препроцессинга, где используется удаление стоп-слов и/или удаление эмоджи (например, когда комментарий изначально состоял только из эмоджи)

In [144]:
def load_data(filepath):
    df = pd.read_csv(filepath, index_col=0, sep='\t')
    print(f'loaded from {filepath}')
    print(df['разметка'].value_counts())
    return df

In [145]:
data = load_data('../Data/Test_data_preprocessed.tsv')
data.fillna('none',  inplace=True)
assert data.isnull().values.any() == False

loaded from ../Data/Test_data_preprocessed.tsv
negative    436
positive    367
neutral     321
speech      189
Name: разметка, dtype: int64


Функция для кодирования разметки в числовые классы. **NB**: классы `neutral` и `speech` объединяются (чтобы было больше данных)

In [146]:
def encode_labeling(label):
    if label == 'negative':
        label_id = -1
    elif label == 'neutral':
        label_id = 0
    elif label == 'positive':
        label_id = 1
    elif label == 'speech':
        label_id = 0
        
    return label_id

Самая обычная tf-idf векторизация, т.к. данных мало и объем вокабуляра не превышает 5к слов. Для ускорения работы используется `analyzer=str.split`, чтобы отключить встроенную токенизацию (т.к. все наши данные и так токенизированны через пробел)

In [147]:
def transform_data(data, col_name):
    y = data['разметка'].apply(encode_labeling)
    
    corpus = data[col_name]
    
    vectorizer = TfidfVectorizer(analyzer=str.split)
    X = vectorizer.fit_transform(corpus)
    
    return X, y

In [205]:
prep_id = str(0)
X, y = transform_data(data, prep_id)
print(X.shape)
original_meta[prep_id]

(1313, 4335)


{'punctuation_deletion': 'no',
 'ner_processing': 'no',
 'lemmatization': 'no',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'no'}

# Модели

Мы решили сравнить 5 алгоритмов класификации

In [214]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier


SEED = 42

In [150]:
def fit_pred(X_train, X_test, y_train, y_test, clf=None):
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    print_report = classification_report(y_test, pred)
    report = classification_report(y_test, pred, output_dict=True)
    #print(print_report)
    return pred, print_report, report

Гридсерч используется ниже, уже после определения наиболее удачной модели и препроцессинга

In [216]:
def grid_pred(X_train, X_test, y_train, y_test, clf=None, params={}, cv=3):
    grid = GridSearchCV(clf, params)
    grid.fit(X_train, y_train)
    #print(grid.best_params_)
    
    best = g_forest.best_estimator_
    
    best.fit(X_train, y_train)
    pred = best.predict(X_test)
    print_report = classification_report(y_test, pred)
    report = classification_report(y_test, pred, output_dict=True)
    #print(print_report)
    
    return best, pred, print_report, report

Сама функция которая обучает все модели и возвращает отчет с метриками

In [152]:
def try_models(X_train, X_test, y_train, y_test, log=False):
    
    clf_logreg = LogisticRegression(random_state=SEED, solver='liblinear')
    clf_sgd = SGDClassifier(random_state=SEED)
    clf_forest = RandomForestClassifier(random_state=SEED)
    clf_bayes = MultinomialNB()
    clf_xgboost = XGBClassifier()

    clfs = {'logreg': {'model': clf_logreg, 'report': {}},
            'sgd': {'model': clf_sgd, 'report': {}},
            'forest': {'model': clf_forest, 'report': {}}, 
            'bayes':  {'model': clf_bayes,  'report': {}},
            'xgboost':  {'model': clf_xgboost,  'report': {}}
           }
    
    for name, clf in clfs.items():
        model = clf['model']
        _, print_report, report = fit_pred(X_train, X_test, y_train, y_test, clf=model)
        clfs[name]['report'] = report
        clfs[name]['pred'] = pred
        
        if log:
            print(name.upper())
            print(print_report)
        
    return clfs

Пример использования на данных, заданных выше

In [206]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
report = try_models(X_train, X_test, y_train, y_test, log=True)

LOGREG
              precision    recall  f1-score   support

          -1       0.70      0.56      0.62       131
           0       0.59      0.76      0.66       153
           1       0.82      0.68      0.74       110

    accuracy                           0.67       394
   macro avg       0.70      0.67      0.67       394
weighted avg       0.69      0.67      0.67       394

SGD
              precision    recall  f1-score   support

          -1       0.68      0.64      0.66       131
           0       0.64      0.70      0.67       153
           1       0.79      0.75      0.77       110

    accuracy                           0.69       394
   macro avg       0.70      0.70      0.70       394
weighted avg       0.70      0.69      0.69       394

FOREST
              precision    recall  f1-score   support

          -1       0.72      0.47      0.56       131
           0       0.56      0.74      0.64       153
           1       0.67      0.66      0.67       110

  

In [208]:
print('weighted avg f1-score\n')

for name, clf in report.items():
    print(f"{name.upper()}\t{clf['report']['weighted avg']['f1-score']}")
          
original_meta[prep_id]

weighted avg f1-score

LOGREG	0.6704129036937323
SGD	0.6935621439388309
FOREST	0.6225348941498408
BAYES	0.6416431412385212
XGBOOST	0.6579754002342835


{'punctuation_deletion': 'no',
 'ner_processing': 'no',
 'lemmatization': 'no',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'no'}

## Исследование моделей с базовыми параметрами и всех препроцессингов

In [153]:
import numpy as np
from tqdm.auto import tqdm

Здесь обучается и замеряется кач-во 960 моделей (за ~8 минут)

In [164]:
for m_id, m in tqdm(meta.items()):
    X, y = transform_data(data, m_id)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
    
    reports = try_models(X_train, X_test, y_train, y_test)
    
    meta[m_id]['reports'] = reports

В качестве метрики выбрали weighted average f1-score, вырезаем только его, чтобы можно было внимательнее посмотреть на результаты

In [173]:
weighted_fs = []
preps = []
models = []

for m_id, m in meta.items():
    #print(original_meta[m_id])
    
    for name, clf in m['reports'].items():
        score = clf['report']['weighted avg']['f1-score']
        
        weighted_fs.append(score)
        preps.append(m_id)
        models.append(name)
        #print(f"{name.upper()}\t{score}")
        
    #print('\n===========\n')

Все варианты с метрикой выше `0.73`:

In [193]:
top_prep = []
top_model = []

for i in np.argsort(weighted_fs)[::-1]:
    if weighted_fs[i] < 0.73:
        break
    top_prep.append(preps[i])
    top_model.append(models[i])
    print(f'{preps[i]}\t{models[i]}\t{weighted_fs[i]}')

129	sgd	0.7460955019649301
151	sgd	0.7445162556013984
182	sgd	0.7437483242722889
117	sgd	0.7417101203331323
183	sgd	0.7416138595677357
97	sgd	0.7410744344834481
176	sgd	0.7393389329607376
150	sgd	0.7386623809291071
87	sgd	0.7364395418749176
181	sgd	0.7363485156033366
113	sgd	0.7362855632878136
83	forest	0.7359792063880859
119	sgd	0.7343673197510162
112	sgd	0.7340846174058158
149	sgd	0.7340678003956387
149	forest	0.7339013202490693
83	sgd	0.7338495227082659
148	sgd	0.7338127762128792
135	sgd	0.7335916200738946
151	logreg	0.7334049279113447
183	logreg	0.7334049279113447
119	logreg	0.7334049279113447
145	sgd	0.7333381483000007
184	sgd	0.7323146816768177
116	sgd	0.7318931263033746
167	sgd	0.731640052754087
54	sgd	0.7316054793737272
119	forest	0.7312113465824605
103	sgd	0.7311894399955903
5	sgd	0.7310752132005299
21	sgd	0.7310752132005299
51	logreg	0.730983563349178
83	logreg	0.730983563349178


Разброс значений для наилучшего алгоритма:

In [198]:
for i in np.argsort(weighted_fs)[::-1]:
    if models[i] == 'sgd':
        print(f'{preps[i]}\t{models[i]}\t{weighted_fs[i]}')

129	sgd	0.7460955019649301
151	sgd	0.7445162556013984
182	sgd	0.7437483242722889
117	sgd	0.7417101203331323
183	sgd	0.7416138595677357
97	sgd	0.7410744344834481
176	sgd	0.7393389329607376
150	sgd	0.7386623809291071
87	sgd	0.7364395418749176
181	sgd	0.7363485156033366
113	sgd	0.7362855632878136
119	sgd	0.7343673197510162
112	sgd	0.7340846174058158
149	sgd	0.7340678003956387
83	sgd	0.7338495227082659
148	sgd	0.7338127762128792
135	sgd	0.7335916200738946
145	sgd	0.7333381483000007
184	sgd	0.7323146816768177
116	sgd	0.7318931263033746
167	sgd	0.731640052754087
54	sgd	0.7316054793737272
103	sgd	0.7311894399955903
5	sgd	0.7310752132005299
21	sgd	0.7310752132005299
53	sgd	0.7291973418044168
118	sgd	0.7289064666228796
33	sgd	0.7281863707333714
161	sgd	0.7281350042485091
144	sgd	0.7266779819764344
133	sgd	0.7264029683632327
101	sgd	0.7263424585141931
164	sgd	0.7262493687767182
180	sgd	0.7261169771682464
177	sgd	0.7259680872317501
178	sgd	0.7258007141477278
120	sgd	0.7244147408495858
166	sgd	0.7

Можно заметить, что лучший результат дает препроцессинг с удалением именованных сущностей, заменой мата на плейсходер и удалением пунктуации:

In [203]:
original_meta['129']

{'punctuation_deletion': 'yes',
 'ner_processing': 'del',
 'lemmatization': 'no',
 'stopwords_deletion': 'no',
 'emojis_processing': 'no',
 'vulgar_processing': 'yes'}

А худший результат получается при удалении всего, что можно, и без нормализации (нет ни лемматизации, ни замены мата):

In [201]:
original_meta['138']

{'punctuation_deletion': 'yes',
 'ner_processing': 'del',
 'lemmatization': 'no',
 'stopwords_deletion': 'yes',
 'emojis_processing': 'del',
 'vulgar_processing': 'no'}

Посмотрим внимательнее на варианты с результатами `> 0.73`

In [177]:
from collections import Counter

In [210]:
c_models = Counter(top_model).most_common()
c_models

[('sgd', 25), ('logreg', 5), ('forest', 3)]

In [179]:
Counter(top_prep).most_common()

[('83', 3),
 ('119', 3),
 ('151', 2),
 ('183', 2),
 ('149', 2),
 ('129', 1),
 ('182', 1),
 ('117', 1),
 ('97', 1),
 ('176', 1),
 ('150', 1),
 ('87', 1),
 ('181', 1),
 ('113', 1),
 ('112', 1),
 ('148', 1),
 ('135', 1),
 ('145', 1),
 ('184', 1),
 ('116', 1),
 ('167', 1),
 ('54', 1),
 ('103', 1),
 ('5', 1),
 ('21', 1),
 ('51', 1)]

In [181]:
original_meta['83']

{'punctuation_deletion': 'no',
 'ner_processing': 'replace',
 'lemmatization': 'yes',
 'stopwords_deletion': 'no',
 'emojis_processing': 'del',
 'vulgar_processing': 'yes'}

In [182]:
original_meta['119']

{'punctuation_deletion': 'yes',
 'ner_processing': 'no',
 'lemmatization': 'yes',
 'stopwords_deletion': 'no',
 'emojis_processing': 'label',
 'vulgar_processing': 'yes'}

Для дальнейшего изучения на всякий случай сохраняем все результаты в файл

In [192]:
json_meta = {}

for m_id, m in meta.items():
    json_meta[m_id] = {'prep': {},
                       'reports': {'logreg': {},
                                   'sgd': {},
                                   'forest': {}, 
                                   'bayes':  {},
                                   'xgboost':  {}}
                      }

for m_id, m in meta.items():
    json_meta[m_id]['prep'] = original_meta[m_id]
    for name, clf in m['reports'].items():
        json_meta[m_id]['reports'][name] = clf['report']

with open('compare.json', 'w') as f:
    json.dump(json_meta, f, ensure_ascii=False, indent=4)

### Подбор параметров

В кач-ве данных возьмем препроцессинги, показавшие наилучший результат для каждой из 3 моделей

In [212]:
for m in c_models:
    for i in np.argsort(weighted_fs)[::-1]:
        if models[i] == m[0]:
            print(f'{preps[i]}\t{models[i]}\t{weighted_fs[i]}')
            break

129	sgd	0.7460955019649301
151	logreg	0.7334049279113447
83	forest	0.7359792063880859


In [ ]:
grid_logreg = LogisticRegression(random_state=SEED, solver='liblinear')
grid_sgd = SGDClassifier(random_state=SEED)
grid_forest = RandomForestClassifier(random_state=SEED)

grid_pipeline = {
    'sgd': {'model': grid_sgd, 'best_model': None, 'best_params': {}, 'report': {}, 'grid':{
        
    }},
    'logreg': {'model': grid_logreg, 'best_model': None, 'best_params': {}, 'report': {}, 'grid':{
        
    }},
    'forest': {'model': grid_forest, 'best_model': None, 'best_params': {}, 'report': {}, 'grid':{
        
    }},
}

In [217]:
best, pred, print_report, report = grid_pred(X_train, X_test, y_train, y_test, clf=None, params={})

TypeError: estimator should be an estimator implementing 'fit' method, None was passed

*Анна Полянская, 2020*